In [1]:
import cv2
import numpy as np

import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from estimator import TfPoseEstimator
from networks import get_graph_path, model_wh

from random import shuffle
from tqdm import tqdm

In [2]:
os.getcwd()

'/Users/nishant/Documents/Intern/GitLab Projects/computervision/tf-openpose/src'

In [3]:
# old_path = os.path.dirname(os.path.abspath(__file__))
old_path = os.getcwd()
# MODEL = 'mobilenet_thin'
# resolution = '432x368'
# w, h = model_wh(resolution)

In [4]:
# e = TfPoseEstimator(get_graph_path(MODEL), target_size=(w, h))
# humans = e.inference(image)
# blank_image = np.zeros((h,w,3), np.uint8)
# image = TfPoseEstimator.draw_humans(blank_image, humans, imgcopy=False)

In [5]:
# cv2.imshow('tf-pose-estimation result', image)
# cv2.waitKey(0)
# if cv2.waitKey(1) & 0xFF == ord('q'):
#     cv2.destroyAllWindows()
# cv2.imwrite('RESULT.jpg', image)

In [6]:
pwd

'/Users/nishant/Documents/Intern/GitLab Projects/computervision/tf-openpose/src'

In [7]:
TRAIN_DIR = '/Users/nishant/DataSet/train/test_img'
# TRAIN_SITTING_DIR = '/Users/nishant/DataSet/train/sitting'
# TRAIN_STANDING_DIR = '/Users/nishant/DataSet/train/standing'
# TRAIN_SLEEPING_DIR = '/Users/nishant/DataSet/train/sleeping'
# TRAIN_UNKNOWN_DIR = '/Users/nishant/DataSet/train/unknown'
TEST_DIR = '/Users/nishant/DataSet/test'

LR=1e-3

In [8]:
def label_img(img):
    word_label = img.split('_')[0]
  
    if word_label == 'sitting': 
        return [1,0,0,0]
    elif word_label == 'standing':
        return [0,1,0,0]
    else:
        return [0,0,1,0]
    
#         return [0,0,0,1]
#     elif word_label == 'sleeping':
#         return [0,0,1,0]
      

In [9]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        # print(path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (50,50))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    #np.save('train_data.npy', training_data)
    return training_data

In [10]:
train_data= create_train_data()

100%|██████████| 120/120 [00:00<00:00, 1511.53it/s]


In [11]:
# import tensorflow as tf
import tflearn

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [12]:
convnet = input_data(shape=[None,50,50,1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet,4, activation='linear')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [13]:
#os.chdir('..')
#os.chdir('./models/graph/mobilenet_thin/')
# cmu = './models/graph/cmu/graph_opt.pb'
# mobilenet_thin = './models/graph/mobilenet_thin/graph_opt.pb'

#if os.path.exists('{}.pb'. format(MODEL)):
 #   model.load(MODEL)
    #print('model loaded!')
#os.chdir(old_path)

In [14]:
train = train_data[:-5]
test = train_data[-5:]

In [15]:
# np.shape(test_x)

In [16]:
X = np.array([i[0] for i in train]).reshape(-1,50,50,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,50,50,1)
test_y = [i[1] for i in test]


In [17]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=20, show_metric=True) #, run_id=model)

Training Step: 5  | total loss: 14.50120 | time: 0.158s
| Adam | epoch: 003 | loss: 14.50120 - acc: 0.6258 -- iter: 064/115
Training Step: 6  | total loss: 13.88626 | time: 1.326s
| Adam | epoch: 003 | loss: 13.88626 - acc: 0.6017 | val_loss: 9.21034 - val_acc: 0.4000 -- iter: 115/115
--
